In [1]:
import time

<h1>In-Class Exercises: Dynamic Programming</h1>

This is a set of exercises to help you visualize the concepts introduced in class.

<h2>1. Gridworld </h2>

The exercises are based on the Gridworld game, which comprises the following rules:<br>
- There is a squared grid with $K$ tiles in each orientation. <br>
- The player (agent) is free to move throughout the grid in 4 different directions: North (N), South (S), East (E), and West (W). <br>
- At the boundary tiles, an attempt to move outside the grid results in returning to the current tile. <br>
- There is at least one "final" tile and the game is over when the agent reaches one of them. <br>
- Our goal is to reach a final tile as quick as possible.

<h2> 2.Markov Decision Processes (MDPs) </h2>

The first step is to represent the problem as an MDP with the following components: <br>
- Set of states: $\mathcal S=\{(x,y)\}_{x=1,\dots,K, y=1,\dots,K}$ (each tile is a state) <br>
- Set of actions $\mathcal A=\{\uparrow, \downarrow, \rightarrow, \leftarrow\}$ (the same for each state) <br>
- Discount Factor: $\gamma$
- States $(1,1)$ and $(K,K)$ are (absorbing) final states

Because, in this example, all actions lead deterministically to a single state, we consider: <br>
- Transition Probabilities: $\mathcal P_{ss'}^{a} = \begin{cases} 1, & \text{ if } a(s)=s' \\ 0, & \text{otherwise}\end{cases}$<br>
- Reward: $\mathcal R_s^a = \mathbb{E} [ R_{t+1} |\, S_t=s, A_t=a] = r = -1$ <br>

In [2]:
class MDP:
    def __init__(self, grid_size=3, gamma=1.0, reward=-1):
        self.k = grid_size  # grid side
        self.g = gamma      # discount factor
        self.r = reward     # fixed reward

        # set of states
        self.S  = [(i+1,j+1) for i in range(self.k) for j in range(self.k)]   

        # set of actions
        self.A  = ["N", "S", "E", "W"]                                            

        # definition of expected reward R
        self.R  = { (s, a): 0 if (s==(1,1) or s==(self.k, self.k)) else self.r for s in self.S for a in self.A }                

        # definition of transition probabilities
        self.Tp = { (s, ss, a): 1 if self.action(s,a) == ss else 0 for s in self.S for ss in self.S for a in self.A}  
    
    # support function "action" that help define the transition probabilities Tp
    def action(self, s, a):   
        if s == (1,1):
            return s
        if s == (self.k, self.k):
            return (1,1)
        if a == "N":
            return (max(1, s[0]-1), s[1])
        if a == "S":
            return (min(self.k, s[0]+1), s[1])
        if a == "E":
            return (s[0], min(self.k, s[1]+1))
        if a == "W":
            return (s[0], max(1, s[1]-1))


<h2> 3.Policy Evaluation </h2>

The idea is to use the Dynamic Programming framework. In our case, we will use Bellman expectation equation iteravitely to compute the value function of each state in our Gridworld MDP.

<b><u>Exercise 2:</b></u> Provide the line of code to calculate the current iteration of the Bellman Expectation Equation for each state. Answer directly on the code below.

In [3]:
# Policy Evaluation Algorithm
# Input:  MDP parameters (states S, actions A, rewards R, transition probabilities Tp)
#         policy Pi
#         initial value v0
#         maximum number of iterations max_iter
# Output: list of the max_iter values for the states (last item of the list estimates v_pi)
def policy_eval(mdp, Pi, v0, max_iter=100):
    v = [v0]
    for i in range(1, max_iter+1):
        v.append({})
        for s in mdp.S:
            v[i][s] = sum( Pi[(s,a)] * (mdp.R[(s,a)] + mdp.g*sum(mdp.Tp[(s,ss,a)]*v[i-1][ss] for ss in mdp.S)) for a in mdp.A)
    return v

<h3> 3.1. Evaluating a <b>Random Policy</b> </h3>

Uniform random policy: $\pi(\uparrow |\,\cdot) = \pi(\rightarrow |\,\cdot) = \pi(\downarrow |\,\cdot) = \pi(\leftarrow|\,\cdot) = 0.25$

<b><u>Exercise 3:</b></u> How many iterations do we need to converge to the value of the random policy? (you can test the results by changing the assignment of variable max_iter) Answer: 106 iterations.

In [4]:
# instantiation of an MDP object with default parameters
mdp = MDP() 

# definition of policy pi
randomPi = { (s, a): 1.0/len(mdp.A) for s in mdp.S for a in mdp.A }   

# definition of the state-value function
v0 = {s: 0 for s in mdp.S}

# maximum number of iterations
max_iter=106

# evaluate policy Pi over max_iter iterations and store value functions of every iteration
v = policy_eval(mdp, randomPi, v0, max_iter)

# take the values of the last iteration, i.e., the best estimate of the value of pi
vpi = v[-1]

## print results
print("These are the values of the random policy")
for s in mdp.S:
    print(" %.5f" %vpi[s] if vpi[s]==0 else "%.5f" %vpi[s], end='\n' if s[1] % mdp.k == 0 else '\t')

These are the values of the random policy
 0.00000	-7.00000	-9.00000
-7.00000	-8.00000	-7.00000
-9.00000	-7.00000	 0.00000


<h3> 3.2. Evaluating an <b>Oracle Policy</b> </h3>

<b><u>Exercise 4:</b></u> Given that we, as external observers, know which states are important for win the Gridworld game, replace the policy $oraclePi$ below to instruct the agent to take the decision you judge to be the best one for every state.

In [5]:
# instantiation of an MDP object with default parameters
mdp = MDP() 

# definition of custom policy pi
# TODO: The policy below behaves exactly like the random policy. Replace the probability of each action with a value you think suits best
oraclePi = {
    ((1,1), "N"): 1.0,
    ((1,1), "S"): 0.0,
    ((1,1), "E"): 0.0,
    ((1,1), "W"): 0.0,

    ((1,2), "N"): 0.0,
    ((1,2), "S"): 0.0,
    ((1,2), "E"): 0.0,
    ((1,2), "W"): 1.0,

    ((1,3), "N"): 0.0,
    ((1,3), "S"): 0.0,
    ((1,3), "E"): 0.0,
    ((1,3), "W"): 1.0,

    ((2,1), "N"): 1.0,
    ((2,1), "S"): 0.0,
    ((2,1), "E"): 0.0,
    ((2,1), "W"): 0.0,

    ((2,2), "N"): 1.0,
    ((2,2), "S"): 0.0,
    ((2,2), "E"): 0.0,
    ((2,2), "W"): 0.0,

    ((2,3), "N"): 0.0,
    ((2,3), "S"): 1.0,
    ((2,3), "E"): 0.0,
    ((2,3), "W"): 0.0,

    ((3,1), "N"): 0.0,
    ((3,1), "S"): 0.0,
    ((3,1), "E"): 1.0,
    ((3,1), "W"): 0.0,

    ((3,2), "N"): 0.0,
    ((3,2), "S"): 0.0,
    ((3,2), "E"): 1.0,
    ((3,2), "W"): 0.0,

    ((3,3), "N"): 1.0,
    ((3,3), "S"): 0.0,
    ((3,3), "E"): 0.0,
    ((3,3), "W"): 0.0,
}

# definition of the state-value function
v0 = {s: 0 for s in mdp.S}

# maximum number of iterations
max_iter=300

# evaluate policy Pi over max_iter iterations and store value functions of every iteration
v = policy_eval(mdp, oraclePi, v0, max_iter)

# take the values of the last iteration, i.e., the best estimate of the value of pi
vpi = v[-1]

## print results
print("These are the values of the oracle policy")
for s in mdp.S:
    print(" %.5f" %vpi[s] if vpi[s]==0 else "%.5f" %vpi[s], end='\n' if s[1] % mdp.k == 0 else '\t')

These are the values of the oracle policy
 0.00000	-1.00000	-2.00000
-1.00000	-2.00000	-1.00000
-2.00000	-1.00000	 0.00000


<h2> 4. Policy Iteration </h2>

In Policy Iteration, we (i) evaluate and (ii) greedily improve the initial policy.<br>
We iterate until we observe convergence ($\epsilon$-convergence) or until we reach the maximum number of iterations ($maxIter$). <br>
Before we see how Policy Iteration works, since we have already covered part (i) Policy Evaluation, let's discuss part (ii) Policy Improvement.

 <h3> 4.1. Policy Improvement </h3>

 The idea of policy improvement is to find a new policy $\pi'$ based on an initial policy $\pi$, such that $v_{\pi'}(s) \geq v_{\pi}(s), \forall s\in S$. 
 The simplest way of doing that is through greedy improvements of the initial policy.
 If we evaluate the value function of all states of the MDP according to the initial policy, i.e., $v_{\pi}$, we can define the $\pi'$ such that the agent`s action is to choose the state with the highest value function.

In [6]:
# Policy Improvement Algorithm
# Input:  MDP parameters (states S, actions A)
#         initial policy
# Output: improved policy (deterministic greedy action)
def policy_improv(mdp, v):
    newPi = {}
    for s in mdp.S:
        best_v = float('-inf')
        best_action = "N"
        for a in mdp.A:
            neighbor = mdp.action(s, a)
            if v[neighbor] >= best_v:
                best_v = v[neighbor]
                best_action = a
        for a in mdp.A:
            newPi[s, a] = 1.0 if a == best_action else 0.0
    return newPi

Now, let's test the policy improvement function using the random policy as our initial policy.

<b><u>Exercise 5:</b></u> Fill the lines for variables $Pi new$ and $v new$ in order to obtain the improved values of the original random policy.

In [7]:
# instantiation of an MDP object with default parameters
mdp = MDP() 

# initial (random) policy
Pi_old = { (s, a): 1.0/len(mdp.A) for s in mdp.S for a in mdp.A }   

# find the value of the initial policy
v_old = policy_eval(mdp, Pi_old, v0 = {s: 0 for s in mdp.S}, max_iter=300)[-1]

# compute new policy based on the improvement of the initial policy
Pi_new = policy_improv(mdp, v_old)

# find the value of the improved policy
v_new = policy_eval(mdp, Pi_new, v0 = {s: 0 for s in mdp.S}, max_iter=300)[-1]

## print results
print("These are the values of the initial policy")
for s in mdp.S:
    print(" %.5f" %v_old[s] if v_old[s]==0 else "%.5f" %v_old[s], end='\n' if s[1] % mdp.k == 0 else '\t')

print("\nThese are the values of the improved policy")
for s in mdp.S:
    print(" %.5f" %v_new[s] if v_new[s]==0 else "%.5f" %v_new[s], end='\n' if s[1] % mdp.k == 0 else '\t')

These are the values of the initial policy
 0.00000	-7.00000	-9.00000
-7.00000	-8.00000	-7.00000
-9.00000	-7.00000	 0.00000

These are the values of the improved policy
 0.00000	-1.00000	-2.00000
-1.00000	-2.00000	-1.00000
-2.00000	-1.00000	 0.00000


<h3> 4.2. Policy Iteration function </h3>
Finally, we have all the elements to implement the Policy Iteration Algorithm as a Python function.

<b><u>Exercise 6:</b></u> Fill the code below to complete the logic of the Policy Iteration algorithm.

In [8]:
# Policy Iteration Algorithm
# Input:  MDP parameters (states S, actions A)
#         initial policy, Pi_initial
#         maximum number of iterations, max_Iter
# Output: (estimate of the) optimal policy
def policy_iteration(mdp, Pi_initial, max_iter):
    Pi_opt = Pi_initial
    for i in range(max_iter):
        v = policy_eval(mdp, Pi_opt, v0 = {s: 0 for s in mdp.S}, max_iter=300)[-1]
        Pi_opt = policy_improv(mdp, v)
    return Pi_opt


<h3> 4.3. Testing Policy Iteration on a more complex Gridworld </h3>

Consider a new version of the $K$-Gridworld where the 2 final states are now granting the following rewards:<br>
- State $(1,1)$ is a pure-win state, where $r=0$, as before; and <br>
- State $(K,K)$ is a damaged-win state, where $r=-4K$.

If the agent reaches state $(K,K)$, it is penalized by $K$ units and it finishes the episode by moving directly to the absorbing state $(1,1)$.
We can change by simply updating the rewards as follows:

In [9]:
mdp = MDP(grid_size=3, gamma=1.0, reward=-1)
for a in mdp.A:
    mdp.R[((mdp.k, mdp.k), a)] = -4*mdp.k

# initial (random) policy
Pi_init = { (s, a): 1.0/len(mdp.A) for s in mdp.S for a in mdp.A } 

# evaluate random policy
v_init = policy_eval(mdp, Pi_init, v0 = {s: 0 for s in mdp.S}, max_iter=300)[-1]

# print value of random policy
print("These are the values of the initial random policy")
for s in mdp.S:
    print(" %.5f" %v_init[s] if v_init[s]==0 else "%.5f" %v_init[s], end='\n' if s[1] % mdp.k == 0 else '\t')

# compute 1-step improved policy
Pi_impr = policy_improv(mdp, v_init)

# evaluate 1-step improved policy
v_impr = policy_eval(mdp, Pi_impr, v0 = {s: 0 for s in mdp.S}, max_iter=300)[-1]

# print value of 1-step improved policy
print("\nThese are the values of the 1-step improved policy")
for s in mdp.S:
    print(" %.5f" %v_impr[s] if v_impr[s]==0 else "%.5f" %v_impr[s], end='\n' if s[1] % mdp.k == 0 else '\t')

# compute optimal policy by Policy Iteration
Pi_opt = policy_iteration(mdp, Pi_init, max_iter=300)

# evaluate optimal policy
v_opt = policy_eval(mdp, Pi_opt, v0 = {s: 0 for s in mdp.S}, max_iter=300)[-1]

# print value of 1-step improved policy
print("\nThese are the values of the optimal policy")
for s in mdp.S:
    print(" %.5f" %v_opt[s] if v_opt[s]==0 else "%.5f" %v_opt[s], end='\n' if s[1] % mdp.k == 0 else '\t')

These are the values of the initial random policy
 0.00000	-11.00000	-15.00000
-11.00000	-14.00000	-15.00000
-15.00000	-15.00000	-12.00000

These are the values of the 1-step improved policy
 0.00000	-1.00000	-2.00000
-1.00000	-2.00000	-13.00000
-2.00000	-13.00000	-12.00000

These are the values of the optimal policy
 0.00000	-1.00000	-2.00000
-1.00000	-2.00000	-3.00000
-2.00000	-3.00000	-12.00000


<h2> 5. Value Iteration </h2>

Value Iteration is a second method to "solve" MDPs using Dynamic Programming. The idea is to iteratively apply Bellman Optimality Equation, which computes the states' values by considering greedy decisions. At the end of the process, we obtain the optimal value of each state and, consequently, the optimal policy.

<h3> 5.1. Value Iteration Function </h3>

<b><u>Exercise 7:</b></u> Provide the line of code to calculate the current iteration of the Bellman Optimality Equation for each state. Answer directly on the code below.

In [10]:
# Value Iteration Algorithm
# Input:  MDP parameters (states S, actions A)
# Output: (Estimate of the) Optimal values
def value_iteration(mdp, max_iter=100):
    v = [{s: 0 for s in mdp.S}]
    for i in range(1, max_iter+1):
        v.append({})
        for s in mdp.S:
            v[i][s] = max((mdp.R[(s,a)] + mdp.g*sum(mdp.Tp[(s,ss,a)]*v[i-1][ss] for ss in mdp.S)) for a in mdp.A)
    return v

<h3> 5.2. Testing Value Iteration </h3>

<b><u>Exercise 8:</b></u> Find the minimium number of iterations for both Policy Iteration (PI) and Value Iteration (VI) to convert to the optimal solution. Observe the differences between number of iterations and runtime between both methods.

In [11]:
num_of_iterations_PI = 5
num_of_iterations_VI = 9

mdp5 = MDP(grid_size=6, gamma=1.0, reward=-1)
for a in mdp5.A:
    mdp5.R[((mdp5.k, mdp5.k), a)] = -4*mdp5.k

# initial (random) policy
Pi_init = { (s, a): 1.0/len(mdp5.A) for s in mdp5.S for a in mdp5.A } 

# compute optimal policy by Policy Iteration
pi_start = time.time()
Pi_PI = policy_iteration(mdp5, Pi_init, max_iter=num_of_iterations_PI)
pi_end = time.time()

# evaluate optimal policy
v_PI = policy_eval(mdp5, Pi_PI, v0 = {s: 0 for s in mdp5.S}, max_iter=300)[-1]

# compute optimal value for the MDP
vi_start = time.time()
v_VI = value_iteration(mdp5, max_iter=num_of_iterations_VI)[-1]
vi_end = time.time()

# print optimal value obtained by Policy Iteration
print("Policy Iteration took %f s." %(pi_end - pi_start))
print("These are its values after %d iterations:" %num_of_iterations_PI)
for s in mdp5.S:
    print(" %.5f" %v_PI[s] if v_PI[s]==0 else "%.5f" %v_PI[s], end='\n' if s[1] % mdp5.k == 0 else '\t')

# print optimal value obtained by Value Iteration
print("Value Iteration took %f s." %(vi_end - vi_start))
print("These are its values after %d iterations:" %num_of_iterations_VI)
for s in mdp5.S:
    print(" %.5f" %v_VI[s] if v_VI[s]==0 else "%.5f" %v_VI[s], end='\n' if s[1] % mdp5.k == 0 else '\t')

Policy Iteration took 0.952484 s.
These are its values after 5 iterations:
 0.00000	-1.00000	-2.00000	-3.00000	-4.00000	-5.00000
-1.00000	-2.00000	-3.00000	-4.00000	-5.00000	-6.00000
-2.00000	-3.00000	-4.00000	-5.00000	-6.00000	-7.00000
-3.00000	-4.00000	-5.00000	-6.00000	-7.00000	-8.00000
-4.00000	-5.00000	-6.00000	-7.00000	-8.00000	-9.00000
-5.00000	-6.00000	-7.00000	-8.00000	-9.00000	-24.00000
Value Iteration took 0.005910 s.
These are its values after 9 iterations:
 0.00000	-1.00000	-2.00000	-3.00000	-4.00000	-5.00000
-1.00000	-2.00000	-3.00000	-4.00000	-5.00000	-6.00000
-2.00000	-3.00000	-4.00000	-5.00000	-6.00000	-7.00000
-3.00000	-4.00000	-5.00000	-6.00000	-7.00000	-8.00000
-4.00000	-5.00000	-6.00000	-7.00000	-8.00000	-9.00000
-5.00000	-6.00000	-7.00000	-8.00000	-9.00000	-24.00000
